In [18]:
import folium
import pandas as pd 

m = folium.Map(
    location=(24.161259, 120.631192
),
    zoom_start=13,
    tiles="Cartodb dark_matter"
    
)

folium.Marker(
    location=(24.161259, 120.631192),
    popup='site',
    tooltip='site'
).add_to(m)


In [13]:
import pandas as pd
from pathlib import Path

p = Path('data/merged_data/MRT.json')
print('Path ->', p.resolve())
print('Exists ->', p.exists())

if p.exists():
    try:
        df_mrtroute = pd.read_json(p)
    except ValueError:
        df_mrtroute = pd.read_json(p, lines=True)
    print('Loaded rows,cols ->', df_mrtroute.shape)
    display(df_mrtroute.head())
else:
    raise FileNotFoundError(f'MRT.json not found at: {p}')


Path -> C:\DCCG_final project\final-project\data\merged_data\MRT.json
Exists -> True
Loaded rows,cols -> (18, 3)


,欄位名稱,數值,經緯度
0,北屯總站,21818,"24.189132,120.70864"
1,舊社,30414,"24.182289,120.707294"
2,松竹,80013,"24.180807,120.70145"
3,四維國小,53208,"24.171243,120.693295"
4,文心崇德,91029,"24.172194,120.684869"


In [22]:
# 將 MRT 站點加入地圖，並根據「數值」調整點的大小
import pandas as pd
import numpy as np

# 確認 df_mrtroute 存在
if 'df_mrtroute' not in globals():
    raise NameError('df_mrtroute 未定義，請先執行載入 MRT 的 cell')

# 解析經緯度欄位 (格式: 'lat,lon')
coords = df_mrtroute['經緯度'].astype(str).str.split(',', expand=True)
df_mrtroute['lat'] = coords[0].astype(float)
df_mrtroute['lon'] = coords[1].astype(float)

# 取出數值欄並轉為數字
vals = pd.to_numeric(df_mrtroute['數值'], errors='coerce').fillna(0)
min_val, max_val = vals.min(), vals.max()

# 定義映射函數 -> 半徑 pixels
min_radius, max_radius = 4, 18

def scale_radius(v):
    if pd.isna(v):
        v = 0
    if max_val == min_val:
        return (min_radius + max_radius) / 2
    return min_radius + (float(v) - min_val) / (max_val - min_val) * (max_radius - min_radius)

for _, r in df_mrtroute.iterrows():
    radius = scale_radius(r['數值'])
    folium.CircleMarker(
        location=(r['lat'], r['lon']),
        radius=radius,
        color='lightgreen',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.6,
        popup=f"{r['欄位名稱']}<br>數值: {r['數值']}",
        tooltip=r['欄位名稱']
    ).add_to(m)

# 加上圖層控制，顯示並儲存
folium.LayerControl().add_to(m)

m




In [ ]:
import pandas as pd
from pathlib import Path
p2 = Path('data/merged_data/intercity_routes.json')
print('Path ->', p2.resolve())
print('Exists ->', p2.exists())
if p2.exists():
    try:
        df_itcroute = pd.read_json(p2)
    except ValueError:
        df_itcroute = pd.read_json(p2, lines=True)
    print('Loaded rows,cols ->', df_itcroute.shape)
else:
    raise FileNotFoundError(f'intercity_routes.json not found at: {p2}')

import numpy as np
import re
import ast

if 'df_itcroute' not in globals():
    raise NameError('df_itcroute 未定義，請先執行載入 intercity routes 的 cell')

# 如果資料為每條路線帶一個 Stops 陣列，先把 Stops explode 成每個站一列
if 'Stops' in df_itcroute.columns:
    exploded_rows = []
    for _, route in df_itcroute.iterrows():
        stops = route.get('Stops')
        # route may contain nested dicts for names; keep route-level fields we want
        route_info = route.to_dict()
        # remove Stops to avoid nesting into each row
        route_info.pop('Stops', None)
        if not stops:
            # no stops for this route, keep route-level row
            exploded_rows.append(route_info)
            continue
        for s in stops:
            row = route_info.copy()
            # StopName may be dict -> prefer Zh_tw then En then full dict string
            stopname = s.get('StopName')
            if isinstance(stopname, dict):
                stop_name = stopname.get('Zh_tw') or stopname.get('En') or str(stopname)
            else:
                stop_name = stopname
            row['StopName'] = stop_name
            row['StopSequence'] = s.get('StopSequence') or s.get('StopOrder')
            # StopPosition may be nested dict with PositionLat/PositionLon
            pos = s.get('StopPosition') or s.get('Position') or s.get('PositionLatLon')
            lat = None
            lon = None
            if isinstance(pos, dict):
                lat = pos.get('PositionLat') or pos.get('lat') or pos.get('Latitude')
                lon = pos.get('PositionLon') or pos.get('lon') or pos.get('Longitude')
            else:
                # fallback: maybe position is a string 'lat,lon' or list
                if isinstance(pos, (list, tuple)) and len(pos) >= 2:
                    try:
                        lat = float(pos[0]); lon = float(pos[1])
                    except Exception:
                        lat = lon = None
                elif isinstance(pos, str):
                    parts = re.split('[,\s]+', pos.strip())
                    if len(parts) >= 2:
                        try:
                            lat = float(parts[0]); lon = float(parts[1])
                        except Exception:
                            lat = lon = None
            row['Lat'] = float(lat) if lat is not None else np.nan
            row['Lon'] = float(lon) if lon is not None else np.nan
            exploded_rows.append(row)
    df_stops = pd.DataFrame(exploded_rows)
    # replace df_itcroute with exploded stops dataframe for plotting
    df_itcroute = df_stops
else:
    # no Stops column -> try to parse position-like columns as before
    pass

# 若 RouteName/SubRouteName 是 dict，取優先顯示的中文或英文名稱
def _name_to_str(v):
    if pd.isna(v):
        return ''
    if isinstance(v, dict):
        return v.get('Zh_tw') or v.get('Zh-tw') or v.get('Zh-TW') or v.get('En') or next(iter(v.values()), '')
    return str(v)

def _choose_route_name(row):
    rn = row.get('RouteName') if 'RouteName' in row.index else None
    sr = row.get('SubRouteName') if 'SubRouteName' in row.index else None
    rn_s = _name_to_str(rn)
    sr_s = _name_to_str(sr)
    if rn_s and rn_s.strip() != '':
        return rn_s
    if sr_s and sr_s.strip() != '':
        return sr_s
    # fallback: try RouteUID or SubRouteUID or Operator name
    if 'RouteUID' in row and pd.notna(row.get('RouteUID')):
        return str(row.get('RouteUID'))
    if 'SubRouteUID' in row and pd.notna(row.get('SubRouteUID')):
        return str(row.get('SubRouteUID'))
    if 'Operators' in row and isinstance(row.get('Operators'), list) and len(row.get('Operators'))>0:
        op = row.get('Operators')[0]
        if isinstance(op, dict):
            on = op.get('OperatorName')
            if isinstance(on, dict):
                return on.get('Zh_tw') or on.get('En') or str(on)
            return str(on)
    return 'unknown'

df_itcroute['RouteDisplay'] = df_itcroute.apply(_choose_route_name, axis=1)

# 粉色/桃紅色調色盤；每條路線用 hash 分配一個粉色
pink_palette = ['#FFC0CB', '#FFB6C1', '#FF69B4', '#FF1493', '#F48FB1', '#F06292', '#F8BBD0']
def _route_color(key):
    return pink_palette[hash(str(key)) % len(pink_palette)]

# 繪製：以 RouteDisplay 分群，繪製同一路線為同一條線，並標示站點 StopName
group_col = 'RouteDisplay'
for route, g in df_itcroute.groupby(group_col, sort=False):
    # 嘗試依 StopSequence/StopOrder 排序
    if 'StopSequence' in g.columns:
        g_sorted = g.sort_values('StopSequence')
    elif 'StopOrder' in g.columns:
        g_sorted = g.sort_values('StopOrder')
    else:
        g_sorted = g
    # 建立路線經緯度串列 (lat, lon)
    locations = list(zip(g_sorted['Lat'].astype(float), g_sorted['Lon'].astype(float)))
    color = _route_color(route)
    if len(locations) >= 2:
        folium.PolyLine(locations=locations, color=color, weight=3, opacity=0.8, popup=str(route)).add_to(m)
    for _, r in g_sorted.iterrows():
        stop_name = r.get('StopName') if 'StopName' in r.index else ''
        folium.CircleMarker(
            location=(r['Lat'], r['Lon']),
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"{stop_name}<br>{route}",
            tooltip=stop_name
        ).add_to(m)

# 加上圖層控制
folium.LayerControl().add_to(m)

m


Path -> C:\DCCG_final project\final-project\data\merged_data\intercity_routes.json
Exists -> True
Loaded rows,cols -> (523, 10)


NameError: name 'folium' is not defined